<a href="https://colab.research.google.com/github/elisasanzani/MachineLearningProject/blob/main/Zboson_decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

import numpy as np
import os
import xgboost as xgb
import time

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

np.random.seed() # shuffle random seed generator



Getting data from my GitHub

In [17]:
!wget https://raw.githubusercontent.com/elisasanzani/MachineLearningProject/main/Z_boson.csv -O Z_boson.csv

--2023-09-12 13:52:05--  https://raw.githubusercontent.com/elisasanzani/MachineLearningProject/main/Z_boson.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1553367 (1.5M) [text/plain]
Saving to: ‘Z_boson.csv’

Z_boson.csv         100%[===================>]   1.48M  --.-KB/s    in 0.01s   

2023-09-12 13:52:05 (154 MB/s) - ‘Z_boson.csv’ saved [1553367/1553367]



Read data

In [28]:
df = pd.read_csv('Z_boson.csv') # read csv file
df['class'] = df['class'].map({'Zee': 0, 'Zmumu': 1}) # convert into numeric
df.head()

,Unnamed: 0,Run,Event,pt1,eta1,phi1,Q1,pt2,eta2,phi2,Q2,class
0,0,166784,20257329,42.9130,-1.3248,-3.0057,-1,42.2897,-0.6624,0.3485,1,1
1,1,163261,64679856,39.8744,-0.0057,-0.7744,-1,36.3997,1.3710,2.2952,1,0
2,2,165570,196860468,72.9296,-0.4162,0.9198,1,39.5410,-0.2680,-1.4629,1,0
3,3,172163,497791581,33.4436,0.3063,-1.6034,1,27.2395,-1.6182,1.5315,-1,0
4,4,173692,550966077,43.9826,0.4276,-0.3876,-1,29.7856,-0.3080,2.8965,1,1


In [29]:
df = df.drop(['Unnamed: 0'], axis=1) # remove unwanted column with just increasing int
df.head()

,Run,Event,pt1,eta1,phi1,Q1,pt2,eta2,phi2,Q2,class
0,166784,20257329,42.9130,-1.3248,-3.0057,-1,42.2897,-0.6624,0.3485,1,1
1,163261,64679856,39.8744,-0.0057,-0.7744,-1,36.3997,1.3710,2.2952,1,0
2,165570,196860468,72.9296,-0.4162,0.9198,1,39.5410,-0.2680,-1.4629,1,0
3,172163,497791581,33.4436,0.3063,-1.6034,1,27.2395,-1.6182,1.5315,-1,0
4,173692,550966077,43.9826,0.4276,-0.3876,-1,29.7856,-0.3080,2.8965,1,1


In [26]:
df.info() #to look for zeroes or NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Run     20000 non-null  int64  
 1   Event   20000 non-null  int64  
 2   pt1     20000 non-null  float64
 3   eta1    20000 non-null  float64
 4   phi1    20000 non-null  float64
 5   Q1      20000 non-null  int64  
 6   pt2     20000 non-null  float64
 7   eta2    20000 non-null  float64
 8   phi2    20000 non-null  float64
 9   Q2      20000 non-null  int64  
 10  class   20000 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 1.7 MB
